<a href="https://colab.research.google.com/github/danielriosgarza/geneExpr/blob/main/geneExpr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Get Code and files from GitHub**

In [1]:
!git clone https://github.com/danielriosgarza/geneExpr.git

Cloning into 'geneExpr'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 102 (delta 31), reused 76 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (102/102), 4.95 MiB | 17.55 MiB/s, done.
Resolving deltas: 100% (31/31), done.


#### **Install missing packages**

In [2]:
!uv pip install gurobipy
!uv pip install cobra

Using Python 3.11.12 environment at: /usr
Resolved 1 package in 99ms
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠹ Preparing packages... (0/1)
Prepared 1 package in 239ms
Installed 1 package in 10ms
 + gurobipy==12.0.2
Using Python 3.11.12 environment at: /usr
Resolved 35 packages in 310ms
⠙ Preparing packages... (0/9)
⠙ Preparing packages... (0/9)
⠙ Preparing packages... (0/9)
ruamel-yaml ------------------------------ 16.00 KiB/114.97 KiB

#### **Import libraries**

In [3]:
import gurobipy
from gurobipy import Model
import cobra
import numpy as np
import os
import sys

import pandas as pd
import numpy as np

%matplotlib inline
import plotly.io as pio
pio.renderers.default = 'colab'
import plotly.io as pio
pio.renderers.default = 'colab'
import plotly.express as px
import plotly.graph_objects as go



#### **Import the main classes and functions**

In [4]:
sys.path.append(os.path.join(os.getcwd(), 'geneExpr','scripts'))

from parseGenExpData import *

# ***Blautia hydrogenotrophica***

#### **Load the gene expression object**

In [5]:
#path with files
bh_geneFolder = os.path.join(os.path.join(os.getcwd(), 'geneExpr','files', 'bh', 'genes'))
bh_modelFolder = os.path.join(os.path.join(os.getcwd(), 'geneExpr','files', 'bh', 'model'))
bh_gsmm = cobra.io.read_sbml_model(os.path.join(bh_modelFolder, 'bh_final.xml'))

bhGE_obj = GeneExpr(geneFolder = bh_geneFolder,
                tpmFile = 'bh_tpm.txt',
                groupLabels = ['t14', 't32', 't72'],
                groupIDX = [[4,5,6],
                            [7,8,9],
                            [10,11,12]
                            ],
                groupComparison = {('14h', '32h'):'t14vst32_deseq.txt',
                                   ('14h', '72h'): 't14vst72_deseq.txt',
                                   ('32h', '72h'): 't32vst72_deseq.txt'},
                featuresFile =  'bh_BVBRC_features.txt',
                sbmlModel = bh_gsmm,
                species='bh'
                )

#### **Get list of model reactions and reactions in the WC carbon metabolism (Fig. 1A)**

In [6]:
bh_reactionList = np.array([i.id for i in bh_gsmm.reactions])

bh_wc_reactions = []

with open(os.path.join(bh_geneFolder, 'wcReactions.txt')) as f:
    for line in f:
        bh_wc_reactions.append(line.strip())

#### **14h vs 32h**

In [7]:
fig_1, results_df_1 = analyze_group('14h', '32h', bhGE_obj, bh_gsmm, bh_reactionList, bh_wc_reactions, 'Blautia hydrogenotrophica')

results_df_1

,reaction,equation,fold_change,p_value,In_WC_model
0,23DHMPO,"(R)-2,3-Dihydroxy-3-methylpentanoate + Nicotin...",-1.242293,1.000000e+00,False
1,26DAPLLAT,"LL-2,6-Diaminoheptanedioate + 2-Oxoglutarate <...",0.000000,1.000000e+00,False
2,2AHBUTI,(S)-2-Aceto-2-hydroxybutanoate <=> (R)-3-Hydro...,-1.242293,1.000000e+00,False
3,2MBCOATA,2-methylbutanoyl-CoA(4-) + acyl carrier protei...,0.000000,1.000000e+00,False
4,3HAD100,(R)-3-Hydroxydecanoyl-[acyl-carrier protein] -...,0.000000,1.000000e+00,False
...,...,...,...,...,...
1072,POR4,Coenzyme A + 2.0 Oxidized ferredoxin + pyruvat...,-5.131987,2.489187e-28,True
1073,rxn47229,2.0 Oxidized ferredoxin + 2.0 Nicotinamide ade...,3.782469,6.900967e-04,False
1074,EX_cpd00020_e,pyruvate <=>,0.000000,1.000000e+00,False
1075,PYRt2r,proton + pyruvate <=> proton + pyruvate,0.000000,1.000000e+00,False


#### **14h vs 72h**

In [8]:
fig_2, results_df_2 = analyze_group('14h', '72h', bhGE_obj, bh_gsmm, bh_reactionList, bh_wc_reactions, 'Blautia hydrogenotrophica')

results_df_2

,reaction,equation,fold_change,p_value,In_WC_model
0,23DHMPO,"(R)-2,3-Dihydroxy-3-methylpentanoate + Nicotin...",-2.033008,0.011072,False
1,26DAPLLAT,"LL-2,6-Diaminoheptanedioate + 2-Oxoglutarate <...",0.000000,1.000000,False
2,2AHBUTI,(S)-2-Aceto-2-hydroxybutanoate <=> (R)-3-Hydro...,-2.033008,0.011072,False
3,2MBCOATA,2-methylbutanoyl-CoA(4-) + acyl carrier protei...,0.000000,1.000000,False
4,3HAD100,(R)-3-Hydroxydecanoyl-[acyl-carrier protein] -...,-1.665276,0.585947,False
...,...,...,...,...,...
1072,POR4,Coenzyme A + 2.0 Oxidized ferredoxin + pyruvat...,3.257598,0.234450,True
1073,rxn47229,2.0 Oxidized ferredoxin + 2.0 Nicotinamide ade...,0.000000,1.000000,False
1074,EX_cpd00020_e,pyruvate <=>,0.000000,1.000000,False
1075,PYRt2r,proton + pyruvate <=> proton + pyruvate,0.000000,1.000000,False


#### **32h vs 72h**

In [9]:
fig, results_df = analyze_group('32h', '72h', bhGE_obj, bh_gsmm, bh_reactionList, bh_wc_reactions, 'Blautia hydrogenotrophica')

results_df

,reaction,equation,fold_change,p_value,In_WC_model
0,23DHMPO,"(R)-2,3-Dihydroxy-3-methylpentanoate + Nicotin...",0.000000,1.000000e+00,False
1,26DAPLLAT,"LL-2,6-Diaminoheptanedioate + 2-Oxoglutarate <...",0.000000,1.000000e+00,False
2,2AHBUTI,(S)-2-Aceto-2-hydroxybutanoate <=> (R)-3-Hydro...,0.000000,1.000000e+00,False
3,2MBCOATA,2-methylbutanoyl-CoA(4-) + acyl carrier protei...,0.000000,1.000000e+00,False
4,3HAD100,(R)-3-Hydroxydecanoyl-[acyl-carrier protein] -...,-1.616344,6.822842e-01,False
...,...,...,...,...,...
1072,POR4,Coenzyme A + 2.0 Oxidized ferredoxin + pyruvat...,3.982119,6.381119e-07,True
1073,rxn47229,2.0 Oxidized ferredoxin + 2.0 Nicotinamide ade...,-3.079376,1.830581e-02,False
1074,EX_cpd00020_e,pyruvate <=>,0.000000,1.000000e+00,False
1075,PYRt2r,proton + pyruvate <=> proton + pyruvate,0.000000,1.000000e+00,False


# ***Bacteroides thetaiotaomicron***

#### **Load the gene expression object**

In [10]:
#path with files
bt_geneFolder = os.path.join(os.path.join(os.getcwd(), 'geneExpr','files', 'bt', 'genes'))
bt_modelFolder = os.path.join(os.path.join(os.getcwd(), 'geneExpr','files', 'bt', 'model'))
bt_gsmm = cobra.io.read_sbml_model(os.path.join(bt_modelFolder, 'bt_final.xml'))

btGE_obj = GeneExpr(geneFolder = bt_geneFolder,
                tpmFile = 'bt_tpm.txt',
                groupLabels = ['04h', '12h', '36h'],
                groupIDX = [[4,5,6],
                            [7,8,9],
                            [10,11,12]
                            ],
                groupComparison = {('04h', '12h'):'t04vst12_deseq.txt',
                                   ('04h', '36h'): 't04vst36_deseq.txt',
                                   ('12h', '36h'): 't12vst36_deseq.txt'},
                featuresFile =  'bt_BVBRC_features.txt',
                sbmlModel = bt_gsmm,
                rootID = None,
                species='bt'
                )




#### **Get list of model reactions and reactions in the WC carbon metabolism (Fig. 1A)**

In [11]:
bt_reactionList = np.array([i.id for i in bt_gsmm.reactions])

bt_wc_reactions = []

with open(os.path.join(bt_geneFolder, 'wcReactions.txt')) as f:
    for line in f:
        bt_wc_reactions.append(line.strip())

#### **04h vs 12h**

In [12]:
fig, results_df = analyze_group('04h', '12h', btGE_obj, bt_gsmm, bt_reactionList, bt_wc_reactions, 'Bacteroides thetaiotaomicron')

results_df

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning:

divide by zero encountered in log10



,reaction,equation,fold_change,p_value,In_WC_model
0,12ETHDt,"1,2-Ethanediol <=> 1,2-Ethanediol",0.000000,1.000000,False
1,12PPDRt,"(R)-Propane-1,2-diol <=> (R)-Propane-1,2-diol",0.000000,1.000000,False
2,12PPDt,"(S)-Propane-1,2-diol <=> (S)-Propane-1,2-diol",0.000000,1.000000,False
3,23PDE2,"2',3'-Cyclic UMP + H2O --> 3'-UMP + proton",0.865225,1.000000,False
4,23PDE4,"2',3'-Cyclic CMP + H2O --> 3'-cmp + proton",0.865225,1.000000,False
...,...,...,...,...,...
1524,ZNabc,ATP(4-) + H2O + Zinc --> ADP + proton + hydrog...,0.608465,1.000000,False
1525,sink_chols,Choline sulfate <=>,0.000000,1.000000,False
1526,sink_hpyr,Hydroxypyruvate <=>,0.000000,1.000000,False
1527,sink_s,Sulfur <=>,0.000000,1.000000,False


#### **4h vs 36h**

In [13]:
fig, results_df = analyze_group('04h', '36h', btGE_obj, bt_gsmm, bt_reactionList, bt_wc_reactions, 'Bacteroides thetaiotaomicron')

results_df

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning:

divide by zero encountered in log10



,reaction,equation,fold_change,p_value,In_WC_model
0,12ETHDt,"1,2-Ethanediol <=> 1,2-Ethanediol",0.000000,1.000000,False
1,12PPDRt,"(R)-Propane-1,2-diol <=> (R)-Propane-1,2-diol",0.000000,1.000000,False
2,12PPDt,"(S)-Propane-1,2-diol <=> (S)-Propane-1,2-diol",0.000000,1.000000,False
3,23PDE2,"2',3'-Cyclic UMP + H2O --> 3'-UMP + proton",1.413246,0.034355,False
4,23PDE4,"2',3'-Cyclic CMP + H2O --> 3'-cmp + proton",1.413246,0.034355,False
...,...,...,...,...,...
1524,ZNabc,ATP(4-) + H2O + Zinc --> ADP + proton + hydrog...,-3.170875,0.002849,False
1525,sink_chols,Choline sulfate <=>,0.000000,1.000000,False
1526,sink_hpyr,Hydroxypyruvate <=>,0.000000,1.000000,False
1527,sink_s,Sulfur <=>,0.000000,1.000000,False


In [14]:
fig, results_df = analyze_group('12h', '36h', btGE_obj, bt_gsmm, bt_reactionList, bt_wc_reactions, 'Bacteroides thetaiotaomicron')

results_df

,reaction,equation,fold_change,p_value,In_WC_model
0,12ETHDt,"1,2-Ethanediol <=> 1,2-Ethanediol",0.000000,1.000000e+00,False
1,12PPDRt,"(R)-Propane-1,2-diol <=> (R)-Propane-1,2-diol",0.000000,1.000000e+00,False
2,12PPDt,"(S)-Propane-1,2-diol <=> (S)-Propane-1,2-diol",0.000000,1.000000e+00,False
3,23PDE2,"2',3'-Cyclic UMP + H2O --> 3'-UMP + proton",0.579329,1.000000e+00,False
4,23PDE4,"2',3'-Cyclic CMP + H2O --> 3'-cmp + proton",0.579329,1.000000e+00,False
...,...,...,...,...,...
1524,ZNabc,ATP(4-) + H2O + Zinc --> ADP + proton + hydrog...,-3.258186,7.117402e-04,False
1525,sink_chols,Choline sulfate <=>,0.000000,1.000000e+00,False
1526,sink_hpyr,Hydroxypyruvate <=>,0.000000,1.000000e+00,False
1527,sink_s,Sulfur <=>,0.000000,1.000000e+00,False


# ***Roseburia intestinalis***

#### **Load the gene expression object**

In [15]:
#path with files
ri_geneFolder = os.path.join(os.path.join(os.getcwd(), 'geneExpr','files', 'ri', 'genes'))
ri_modelFolder = os.path.join(os.path.join(os.getcwd(), 'geneExpr','files', 'ri', 'model'))
ri_gsmm = cobra.io.read_sbml_model(os.path.join(ri_modelFolder, 'ri_final.xml'))

riGE_obj = GeneExpr(geneFolder = ri_geneFolder,
                tpmFile = 'ri_tpm.txt',
                groupLabels = ['04h', '12h', '48h'],
                groupIDX = [[4,5,6],
                            [7,8,9],
                            [10,11,12]
                            ],
                groupComparison = {('04h', '12h'):'t04vst12_deseq.txt',
                                   ('04h', '48h'): 't04vst48_deseq.txt',
                                   ('12h', '48h'): 't12vst48_deseq.txt'},
                featuresFile =  'ri_BVBRC_features.txt',
                sbmlModel = ri_gsmm,
                rootID = None,
                species='ri'
                )



#### **Get list of model reactions and reactions in the WC carbon metabolism (Fig. 1A)**

In [16]:
ri_reactionList = np.array([i.id for i in ri_gsmm.reactions])

ri_wc_reactions = []

with open(os.path.join(ri_geneFolder, 'wcReactions.txt')) as f:
    for line in f:
        ri_wc_reactions.append(line.strip())

#### **4h vs 12h**

In [17]:
fig, results_df = analyze_group('04h', '12h', riGE_obj, ri_gsmm, ri_reactionList, ri_wc_reactions, 'Roseburia intestinalis')

results_df

,reaction,equation,fold_change,p_value,In_WC_model
0,23DHMPO,"(R)-2,3-Dihydroxy-3-methylpentanoate + Nicotin...",-0.556519,1.0,False
1,23PDE2,"2,3-cyclic UMP(1-) + Water --> 3-UMP(2-) + proton",-0.228089,1.0,False
2,23PDE4,"2,3-Cyclic CMP + Water --> 3-cmp + proton",-0.228089,1.0,False
3,23PDE7,"2,3-Cyclic AMP + Water --> 3-AMP(2-) + proton",-0.228089,1.0,False
4,23PDE9,"2,3-Cyclic GMP + Water --> 3-GMP + proton",-0.228089,1.0,False
...,...,...,...,...,...
1130,r1144,L-glutamate(1-) + Sodium --> L-glutamate(1-) +...,0.080806,1.0,False
1131,r1162,ATP + Water + sulfate --> ADP + proton + hydro...,-4.180038,1.0,False
1132,r2137,Chloride <=> Chloride,0.845233,1.0,False
1133,rtranscription,--> RNA transcription,0.829213,1.0,False


#### **04h vs 48h**

In [18]:
fig, results_df = analyze_group('04h', '48h', riGE_obj, ri_gsmm, ri_reactionList, ri_wc_reactions, 'Roseburia intestinalis')

results_df

,reaction,equation,fold_change,p_value,In_WC_model
0,23DHMPO,"(R)-2,3-Dihydroxy-3-methylpentanoate + Nicotin...",-0.869681,1.000000e+00,False
1,23PDE2,"2,3-cyclic UMP(1-) + Water --> 3-UMP(2-) + proton",0.526044,1.000000e+00,False
2,23PDE4,"2,3-Cyclic CMP + Water --> 3-cmp + proton",0.526044,1.000000e+00,False
3,23PDE7,"2,3-Cyclic AMP + Water --> 3-AMP(2-) + proton",0.526044,1.000000e+00,False
4,23PDE9,"2,3-Cyclic GMP + Water --> 3-GMP + proton",0.526044,1.000000e+00,False
...,...,...,...,...,...
1130,r1144,L-glutamate(1-) + Sodium --> L-glutamate(1-) +...,0.735398,1.000000e+00,False
1131,r1162,ATP + Water + sulfate --> ADP + proton + hydro...,-2.295025,1.004450e-13,False
1132,r2137,Chloride <=> Chloride,0.681113,1.000000e+00,False
1133,rtranscription,--> RNA transcription,-2.006745,3.488534e-10,False


#### **12h vs 48h**

In [19]:
fig, results_df = analyze_group('12h', '48h', riGE_obj, ri_gsmm, ri_reactionList, ri_wc_reactions, 'Roseburia intestinalis')

results_df

,reaction,equation,fold_change,p_value,In_WC_model
0,23DHMPO,"(R)-2,3-Dihydroxy-3-methylpentanoate + Nicotin...",-0.207485,1.000000e+00,False
1,23PDE2,"2,3-cyclic UMP(1-) + Water --> 3-UMP(2-) + proton",0.856280,1.000000e+00,False
2,23PDE4,"2,3-Cyclic CMP + Water --> 3-cmp + proton",0.856280,1.000000e+00,False
3,23PDE7,"2,3-Cyclic AMP + Water --> 3-AMP(2-) + proton",0.856280,1.000000e+00,False
4,23PDE9,"2,3-Cyclic GMP + Water --> 3-GMP + proton",0.856280,1.000000e+00,False
...,...,...,...,...,...
1130,r1144,L-glutamate(1-) + Sodium --> L-glutamate(1-) +...,0.755968,1.000000e+00,False
1131,r1162,ATP + Water + sulfate --> ADP + proton + hydro...,-1.517935,7.286076e-01,False
1132,r2137,Chloride <=> Chloride,-0.071151,1.000000e+00,False
1133,rtranscription,--> RNA transcription,-2.467383,1.483918e-07,False
